In [1]:
import pickle
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from src.xg.clean_data import get_data, convert_data
import numpy as np
import hockey_scraper as hs

Scrape play-by-play data from seasons

In [2]:
hs.scrape_seasons([2008, 2018], if_scrape_shifts=False, dir_path=Path("../data/pbp_csv/"), n_jobs=10)

Scraping Game  2008020067 2008-10-18Scraping Game  2008020001 2008-10-04Scraping Game  2008020034 2008-10-13Scraping Game  2008020265 2008-11-18Scraping Game  2008020165 2008-11-02Scraping Game  2008020199 2008-11-08
Scraping Game  2008020232 2008-11-13




Scraping Game  2008020298 2008-11-23

Scraping Game  2008020133 2008-10-29Scraping Game  2008020100 2008-10-24



Scraping Game  2008020299 2008-11-24
Scraping Game  2008020101 2008-10-24
Scraping Game  2008020134 2008-10-29
Scraping Game  2008020233 2008-11-13
Scraping Game  2008020167 2008-11-02
Scraping Game  2008020002 2008-10-04
Scraping Game  2008020200 2008-11-08
Scraping Game  2008020266 2008-11-18
Scraping Game  2008020068 2008-10-18
Scraping Game  2008020035 2008-10-13
Scraping Game  2008020168 2008-11-03
Scraping Game  2008020300 2008-11-24
Scraping Game  2008020102 2008-10-24Scraping Game  2008020135 2008-10-29

Scraping Game  2008020201 2008-11-08
Scraping Game  2008020234 2008-11-13
Scraping Game  2008020036 2008-10-13
Scraping Game  2008020267 2008-11-18
Scraping Game  2008020003 2008-10-05
Scraping Game  2008020069 2008-10-18
Scraping Game  2008020169 2008-11-03
Scraping Game  2008020202 2008-11-08
Scraping Game  2008020235 2008-11-13
Scraping Game  2008020037 2008-10-13
Scraping Game  2008020004 2008-10-05
Scraping Game  2008020301 2008-11-24
Scraping Game  2008020268 2008-11-19
S

Error: Espn is missing coordinates for game 2008-11-14 CHI STL


Scraping Game  2008020241 2008-11-14Scraping Game  2008020176 2008-11-04

Scraping Game  2008020307 2008-11-25
Scraping Game  2008020208 2008-11-09
Scraping Game  2008020044 2008-10-15
Scraping Game  2008020077 2008-10-19
Scraping Game  2008020109 2008-10-25
Scraping Game  2008020275 2008-11-20
Scraping Game  2008020143 2008-10-30
Scraping Game  2008020177 2008-11-04
Scraping Game  2008020011 2008-10-10
Scraping Game  2008020246 2008-11-15
Scraping Game  2008020308 2008-11-25
Scraping Game  2008020210 2008-11-09
Scraping Game  2008020078 2008-10-20
Scraping Game  2008020045 2008-10-15
Scraping Game  2008020111 2008-10-25
Scraping Game  2008020276 2008-11-20
Scraping Game  2008020178 2008-11-05
Scraping Game  2008020079 2008-10-20
Scraping Game  2008020012 2008-10-10
Scraping Game  2008020309 2008-11-26
Scraping Game  2008020144 2008-10-30
Scraping Game  2008020242 2008-11-15
Scraping Game  2008020209 2008-11-09
Scraping Game  2008020046 2008-10-15
Scraping Game  2008020179 2008-11-05
S

Error: Espn is missing coordinates for game 2008-10-25 CHI DET


...

Scraping Game  2018020190 2018-11-01
Scraping Game  2018020223 2018-11-06
Scraping Game  2018020325 2018-11-21
Scraping Game  2018020055 2018-10-11
Scraping Game  2018020293 2018-11-17
Scraping Game  2018020018 2018-10-06
Scraping Game  2018020123 2018-10-23
Scraping Game  2018020259 2018-11-11
Scraping Game  2018020158 2018-10-27
Scraping Game  2018020089 2018-10-17
Scraping Game  2018020191 2018-11-01
Scraping Game  2018020224 2018-11-07
Scraping Game  2018020326 2018-11-21
Scraping Game  2018020056 2018-10-11
Scraping Game  2018020294 2018-11-17Scraping Game  2018020124 2018-10-23

Scraping Game  2018020019 2018-10-06
Scraping Game  2018020090 2018-10-18
Scraping Game  2018020260 2018-11-12
Scraping Game  2018020159 2018-10-28
Scraping Game  2018020225 2018-11-07
Scraping Game  2018020192 2018-11-02
Scraping Game  2018020327 2018-11-21
Scraping Game  2018020057 2018-10-13
Scraping Game  2018020298 2018-11-17
Scraping Game  2018020125 2018-10-23
Scraping Game  2018020021 2018-10-06
S

Scraping Game  2018020064 2018-10-13
Scraping Game  2018020336 2018-11-23
Scraping Game  2018020199 2018-11-03
Scraping Game  2018020097 2018-10-18
Scraping Game  2018020231 2018-11-08
Scraping Game  2018020029 2018-10-07
Scraping Game  2018020132 2018-10-24
Scraping Game  2018020302 2018-11-17
Scraping Game  2018020268 2018-11-13
Scraping Game  2018020166 2018-10-29
Scraping Game  2018020065 2018-10-13
Scraping Game  2018020098 2018-10-18
Scraping Game  2018020133 2018-10-24
Scraping Game  2018020200 2018-11-03
Scraping Game  2018020232 2018-11-08
Scraping Game  2018020333 2018-11-23
Scraping Game  2018020303 2018-11-18
Scraping Game  2018020030 2018-10-07
Scraping Game  2018020066 2018-10-13
Scraping Game  2018020269 2018-11-13
Scraping Game  2018020167 2018-10-30
Scraping Game  2018020099 2018-10-19
Scraping Game  2018020201 2018-11-03
Scraping Game  2018020334 2018-11-23
Scraping Game  2018020233 2018-11-08
Scraping Game  2018020304 2018-11-18
Scraping Game  2018020134 2018-10-25
S

Scraping Game  2018020645 2019-01-06
Scraping Game  2018020445 2018-12-08
Scraping Game  2018020412 2018-12-02
Scraping Game  2018020614 2019-01-01
Scraping Game  2018020480 2018-12-13
Scraping Game  2018020379 2018-11-28
Scraping Game  2018020344 2018-11-23
Scraping Game  2018020549 2018-12-22
Scraping Game  2018020515 2018-12-17
Scraping Game  2018020582 2018-12-28
Scraping Game  2018020446 2018-12-08
Scraping Game  2018020649 2019-01-06
Scraping Game  2018020615 2019-01-02
Scraping Game  2018020413 2018-12-03
Scraping Game  2018020481 2018-12-13
Scraping Game  2018020380 2018-11-28
Scraping Game  2018020346 2018-11-23
Scraping Game  2018020516 2018-12-17
Scraping Game  2018020583 2018-12-29
Scraping Game  2018020550 2018-12-22
Scraping Game  2018020648 2019-01-06
Scraping Game  2018020414 2018-12-03Scraping Game  2018020482 2018-12-13

Scraping Game  2018020616 2019-01-02
Scraping Game  2018020447 2018-12-08
Scraping Game  2018020347 2018-11-24
Scraping Game  2018020381 2018-11-28
S

Scraping Game  2018020927 2019-02-20
Scraping Game  2018020725 2019-01-16
Scraping Game  2018020826 2019-02-07
Scraping Game  2018020692 2019-01-12
Scraping Game  2018020860 2019-02-11
Scraping Game  2018020894 2019-02-16
Scraping Game  2018020994 2019-03-01
Scraping Game  2018020961 2019-02-24
Scraping Game  2018020792 2019-02-02
Scraping Game  2018020928 2019-02-20
Scraping Game  2018020760 2019-01-22
Scraping Game  2018020726 2019-01-17
Scraping Game  2018020827 2019-02-07
Scraping Game  2018020898 2019-02-16
Scraping Game  2018020693 2019-01-12
Scraping Game  2018020995 2019-03-01
Scraping Game  2018020861 2019-02-11
Scraping Game  2018020962 2019-02-25
Scraping Game  2018020793 2019-02-02
Scraping Game  2018020929 2019-02-21
Scraping Game  2018020761 2019-01-22
Scraping Game  2018020727 2019-01-17
Scraping Game  2018020828 2019-02-07
Scraping Game  2018020694 2019-01-12
Scraping Game  2018020897 2019-02-16
Scraping Game  2018020996 2019-03-01
Scraping Game  2018020862 2019-02-11
S

KeyboardInterrupt: 

Fit model


In [2]:
cv = pickle.load(open("../models/logistic_xg.pkl", "rb"))

In [3]:
data = get_data(Path("../"))

data['Outcome'] = np.where(data['Outcome'] == 0, 0, np.where(data['Outcome'] == 1, 0, np.where(data['Outcome'] == 2, 1, 3)))

data = data[data['Outcome'] != 3]

# Convert to lists
features, labels = convert_data(data)

2007
Rink Adjust
Get previous info
Empty nets
Get New Stuff


In [4]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=.2, random_state=42)

In [5]:
cv.best_estimator_.score(x_train, y_train)

0.9322800465612572

In [6]:
cv.best_estimator_.score(x_test, y_test)

0.9318071587932875